## Carregamento dos arquivos json

In [0]:
df_resumo = spark.read.json("dbfs:/FileStore/_covid_data_lake/_raw/_summary/summary_paises.json")
df_pais = spark.read.json("dbfs:/FileStore/_covid_data_lake/_raw/_pais/pais.json")
df_dados_paises = spark.read.json("dbfs:/FileStore/_covid_data_lake/_raw/_dados_paises/dados_paises.json")

### Resumo estatístico das tabelas dados_paises e resumo (summary)

In [0]:
display(df_resumo.select('NEWCONFIRMED', 'NEWDEATHS', 'NEWRECOVERED', 'TOTALCONFIRMED', 'TOTALDEATHS', 'TOTALRECOVERED')\
        .describe())

summary,NEWCONFIRMED,NEWDEATHS,NEWRECOVERED,TOTALCONFIRMED,TOTALDEATHS,TOTALRECOVERED
count,190,190,190,190,190,190
mean,2461.0105263157893,50.94736842105263,1238.4789473684211,620814.5842105263,13777.857894736842,351794.5789473684
stddev,8162.603986544886,214.61055504553707,3997.9161406673356,2481582.6010728846,49210.16894874296,1173407.4292026402
min,0,0,0,1,0,0
max,79876,2286,43428,29236735,529263,10938146


In [0]:
display(df_dados_paises.select('ACTIVE', 'CONFIRMED', 'DEATHS', 'RECOVERED')\
        .describe())

summary,ACTIVE,CONFIRMED,DEATHS,RECOVERED
count,78660,78660,78660,78660
mean,72665.01670480549,187597.5742817188,4821.271217899822,110917.69894482584
stddev,862209.7110509692,1121032.0270692355,23423.735524453583,596277.59700151
min,0,0,0,0
max,28624522,29236735,529263,10938146


In [0]:
display(df_dados_paises)
display(df_pais)
display(df_resumo)

ACTIVE,CONFIRMED,DATE,DEATHS,ID,ID_PAIS,LAT,LON,RECOVERED
0,0,2020-01-22T00:00:00.000Z,0,1,154,0,0,0
0,0,2020-01-23T00:00:00.000Z,0,2,154,0,0,0
0,0,2020-01-24T00:00:00.000Z,0,3,154,0,0,0
0,0,2020-01-25T00:00:00.000Z,0,4,154,0,0,0
0,0,2020-01-26T00:00:00.000Z,0,5,154,0,0,0
0,0,2020-01-27T00:00:00.000Z,0,6,154,0,0,0
0,0,2020-01-28T00:00:00.000Z,0,7,154,0,0,0
0,0,2020-01-29T00:00:00.000Z,0,8,154,0,0,0
0,0,2020-01-30T00:00:00.000Z,0,9,154,0,0,0
0,0,2020-01-31T00:00:00.000Z,0,10,154,0,0,0


ID,NOME,SIGLA,SLUG
1,Guyana,GY,guyana
2,Netherlands,NL,netherlands
3,Norway,NO,norway
4,San Marino,SM,san-marino
5,Sudan,SD,sudan
6,"Taiwan, Republic of China",TW,taiwan
7,Zimbabwe,ZW,zimbabwe
8,Isle of Man,IM,isle-of-man
9,Luxembourg,LU,luxembourg
10,Trinidad and Tobago,TT,trinidad-and-tobago


DATE,ID,ID_PAIS,NEWCONFIRMED,NEWDEATHS,NEWRECOVERED,TOTALCONFIRMED,TOTALDEATHS,TOTALRECOVERED
2021-03-12T00:23:35.507Z,1,199,18,0,90,55894,2451,49499
2021-03-12T00:23:35.507Z,2,207,631,17,714,114840,1986,77498
2021-03-12T00:23:35.507Z,3,117,138,4,109,114681,3026,79428
2021-03-12T00:23:35.507Z,4,243,41,0,23,11130,112,10708
2021-03-12T00:23:35.507Z,5,63,47,0,84,21161,516,19761
2021-03-12T00:23:35.507Z,6,201,20,1,34,882,23,489
2021-03-12T00:23:35.507Z,7,140,7693,107,5049,2169694,53359,1961640
2021-03-12T00:23:35.507Z,8,26,340,5,163,175538,3237,165259
2021-03-12T00:23:35.507Z,9,195,16,0,9,29090,909,22945
2021-03-12T00:23:35.507Z,10,81,2528,19,2012,481919,8776,449053


In [0]:
df_pais_dados_pais = df_dados_paises.join(df_pais,on=df_dados_paises.ID_PAIS == df_pais.ID)
display(df_pais_dados_pais)

ACTIVE,CONFIRMED,DATE,DEATHS,ID,ID_PAIS,LAT,LON,RECOVERED,ID,NOME,SIGLA,SLUG
0,0,2020-01-22T00:00:00.000Z,0,1,154,0,0,0,154,Lesotho,LS,lesotho
0,0,2020-01-23T00:00:00.000Z,0,2,154,0,0,0,154,Lesotho,LS,lesotho
0,0,2020-01-24T00:00:00.000Z,0,3,154,0,0,0,154,Lesotho,LS,lesotho
0,0,2020-01-25T00:00:00.000Z,0,4,154,0,0,0,154,Lesotho,LS,lesotho
0,0,2020-01-26T00:00:00.000Z,0,5,154,0,0,0,154,Lesotho,LS,lesotho
0,0,2020-01-27T00:00:00.000Z,0,6,154,0,0,0,154,Lesotho,LS,lesotho
0,0,2020-01-28T00:00:00.000Z,0,7,154,0,0,0,154,Lesotho,LS,lesotho
0,0,2020-01-29T00:00:00.000Z,0,8,154,0,0,0,154,Lesotho,LS,lesotho
0,0,2020-01-30T00:00:00.000Z,0,9,154,0,0,0,154,Lesotho,LS,lesotho
0,0,2020-01-31T00:00:00.000Z,0,10,154,0,0,0,154,Lesotho,LS,lesotho


In [0]:
df_pais_dados_pais = df_pais_dados_pais.drop('ID', 'ID_PAIS', 'LAT', 'LON', 'ID')
df_pais_dados_pais.show(7)

+------+---------+--------------------+------+---------+-------+-----+-------+
ACTIVE|CONFIRMED| DATE|DEATHS|RECOVERED| NOME|SIGLA| SLUG|
+------+---------+--------------------+------+---------+-------+-----+-------+
 0| 0|2020-01-22T00:00:...| 0| 0|Lesotho| LS|lesotho|
 0| 0|2020-01-23T00:00:...| 0| 0|Lesotho| LS|lesotho|
 0| 0|2020-01-24T00:00:...| 0| 0|Lesotho| LS|lesotho|
 0| 0|2020-01-25T00:00:...| 0| 0|Lesotho| LS|lesotho|
 0| 0|2020-01-26T00:00:...| 0| 0|Lesotho| LS|lesotho|
 0| 0|2020-01-27T00:00:...| 0| 0|Lesotho| LS|lesotho|
 0| 0|2020-01-28T00:00:...| 0| 0|Lesotho| LS|lesotho|
+------+---------+--------------------+------+---------+-------+-----+-------+
only showing top 7 rows

In [0]:
print(df_pais_dados_pais.dtypes)

[('ACTIVE', 'bigint'), ('CONFIRMED', 'bigint'), ('DATE', 'string'), ('DEATHS', 'bigint'), ('RECOVERED', 'bigint'), ('NOME', 'string'), ('SIGLA', 'string'), ('SLUG', 'string')]

In [0]:
df_pais_dados_pais.printSchema()

root
-- ACTIVE: long (nullable = true)
-- CONFIRMED: long (nullable = true)
-- DATE: string (nullable = true)
-- DEATHS: long (nullable = true)
-- RECOVERED: long (nullable = true)
-- NOME: string (nullable = true)
-- SIGLA: string (nullable = true)
-- SLUG: string (nullable = true)